In [13]:
!nvidia-smi

Wed Aug  7 11:42:18 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               On  |   00000000:3F:00.0 Off |                  Off |
| 30%   27C    P8             22W /  300W |       3MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#creating a bangla tokenizer 


In [ ]:
#training the llama model on the bangla dataset 

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
#download unsloth
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes datasets transformers


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-oy39idnk/unsloth_e38a356a46b7450d8223692acfe7bd7d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-oy39idnk/unsloth_e38a356a46b7450d8223692acfe7bd7d
  Resolved https://github.com/unslothai/unsloth.git to commit 637ed8c6bd252f981e89e30e1085efc03a06a880
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [8]:
# !pip install datasets tokenizers transformers[sentencepiece]

In [15]:
from datasets import load_dataset
from tokenizers.normalizers import NFD, StripAccents, Replace, Sequence
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, trainers, processors, decoders
import re
from tokenizers import Regex
from tokenizers.normalizers import NFD, StripAccents, Replace, Sequence


In [13]:
import wandb
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset
from tokenizers import Tokenizer
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
from huggingface_hub import HfApi, list_models
import torch
from unsloth import FastLanguageModel
from tqdm import tqdm
from huggingface_hub import login
from peft import get_peft_model, LoraConfig

In [9]:
!pip install --no-deps --upgrade "flash-attn>=2.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=188977699 sha256=d16f6008f2b7727718653bfd087a20f3e18e2a4c8e91b9f38da2439f3b0bf529
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn
  Attempting uninstall: flash-attn
    Found existing installation: flash-attn 2.5.5
    Uninstalling flash-attn-2.5.5:
      Successfully uninstalled flash-attn-2.5.5


In [8]:
# load bangla dataset
dataset = load_dataset("oscar", "unshuffled_deduplicated_bn", split="train")

#link to dataset  = https://huggingface.co/datasets/oscar-corpus/oscar

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/1114481 [00:00<?, ? examples/s]

In [9]:
def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

In [10]:
# BPE tokenizer 
tokenizer = Tokenizer(models.BPE())

In [16]:
tokenizer.normalizer = normalizers.Sequence([
    normalizers.NFD(),
    normalizers.StripAccents(),
    normalizers.Replace(Regex(r"[\s]+"), " "),
])


In [17]:
# Set normalizer
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [18]:
# Initialize the trainer
trainer = trainers.BpeTrainer(vocab_size=30000, special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])


In [ ]:
# Train the tokenizer
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

# Save
tokenizer.save("bangla_tokenizer.json")

In [20]:
# load
bangla_tokenizer = Tokenizer.from_file("bangla_tokenizer.json")

In [21]:
# Wrap the tokenizer
bangla_tokenizer = PreTrainedTokenizerFast(tokenizer_object=bangla_tokenizer, unk_token="<unk>", pad_token="<pad>", cls_token="<s>", sep_token="</s>", mask_token="<mask>")

In [30]:
def format_dataset(examples):
    texts = examples["text"]
    # Tokenize the texts
    tokenized_texts = bangla_tokenizer(texts, padding="max_length", truncation=True, max_length=512)
    return {"input_ids": tokenized_texts["input_ids"], "attention_mask": tokenized_texts["attention_mask"]}


In [31]:
# Load and format the dataset
from datasets import load_dataset
dataset = load_dataset("oscar", "unshuffled_deduplicated_bn", split="train[:1%]")
dataset = dataset.map(format_dataset, batched=True)

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/11145 [00:00<?, ? examples/s]

In [24]:
# Initialize the model
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.dev0.
   \\   /|    GPU: NVIDIA RTX A6000. Max memory: 47.529 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.2+cu118. CUDA = 8.6. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.23.post1+cu118. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [32]:
# Attach LoRA Adapters
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, lora_config)

In [33]:
# Define data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=bangla_tokenizer,
    mlm=False
)


In [34]:
training_args = TrainingArguments(
    output_dir="./bangla_llama",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=1,
    evaluation_strategy="steps",
    eval_steps=10_000,
    learning_rate=5e-5,
    weight_decay=0.01,
    report_to="wandb",
    remove_unused_columns=False
)


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [35]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=dataset
)


In [ ]:
# Train the model with progress bar
wandb.init(project="bangla-llama-finetuning")
for epoch in range(training_args.num_train_epochs):
    print(f"Epoch {epoch+1}/{training_args.num_train_epochs}")
    trainer.train()

# Save the model with progress bar
print("Saving the model...")
trainer.save_model("./bangla_llama")
bangla_tokenizer.save("bangla_llama")
print("Model saved.")

# Finish Weights & Biases run
wandb.finish()

In [5]:
# final workign code 

In [ ]:
# import wandb
from datasets import load_dataset
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
from unsloth import FastLanguageModel
from tqdm import tqdm
from huggingface_hub import login
from peft import get_peft_model, LoraConfig

# Initialize Weights & Biases
# wandb.init(project="bangla-llama-finetuning")

# Load the custom Bangla tokenizer
bangla_tokenizer = Tokenizer.from_file("bangla_tokenizer.json")

# Wrap the tokenizer
bangla_tokenizer = PreTrainedTokenizerFast(tokenizer_object=bangla_tokenizer, unk_token="<unk>", pad_token="<pad>", cls_token="<s>", sep_token="</s>", mask_token="<mask>")

# Load the dataset
dataset = load_dataset("oscar", "unshuffled_deduplicated_bn", split="train[:1%]")

# Function to format the dataset for training
def format_dataset(examples):
    texts = examples["text"]
    # Tokenize the texts
    tokenized_texts = bangla_tokenizer(texts, padding="max_length", truncation=True, max_length=512)
    return {"input_ids": tokenized_texts["input_ids"], "attention_mask": tokenized_texts["attention_mask"]}

# Apply tokenization to the dataset
dataset = dataset.map(format_dataset, batched=True)

# Remove unnecessary columns
dataset = dataset.remove_columns(["id", "text"])

# Initialize the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)

# Attach LoRA Adapters
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, lora_config)

# Define data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=bangla_tokenizer,
    mlm=False,
    pad_to_multiple_of=8 
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./bangla_llama",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=1,
    evaluation_strategy="steps",
    eval_steps=10_000,
    learning_rate=5e-5,
    weight_decay=0.01,
    # report_to="wandb",
    remove_unused_columns=False
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=dataset
)

# Train the model with progress bar
for epoch in range(training_args.num_train_epochs):
    print(f"Epoch {epoch+1}/{training_args.num_train_epochs}")
    trainer.train()

# Save the model with progress bar
print("Saving the model...")
trainer.save_model("./bangla_llama")
bangla_tokenizer.save("bangla_llama")
print("Model saved.")

# Finish Weights & Biases run
# wandb.finish()


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
This can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.4.
   \\   /|    GPU: NVIDIA RTX A6000. Max memory: 47.529 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.2+cu118. CUDA = 8.6. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.23.post1+cu118. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Epoch 2/3


Step,Training Loss,Validation Loss


Epoch 3/3


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 11,145 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 1
\        /    Total batch size = 4 | Total steps = 8,361
 "-____-"     Number of trainable parameters = 6,815,744


Step,Training Loss,Validation Loss


In [3]:
# import wandb
from datasets import load_dataset
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
from unsloth import FastLanguageModel
from tqdm import tqdm
from huggingface_hub import login
from peft import get_peft_model, LoraConfig

# Initialize Weights & Biases
# wandb.init(project="bangla-llama-finetuning")

# Load the custom Bangla tokenizer
bangla_tokenizer = Tokenizer.from_file("bangla_tokenizer.json")

# Wrap the tokenizer
bangla_tokenizer = PreTrainedTokenizerFast(tokenizer_object=bangla_tokenizer, unk_token="<unk>", pad_token="<pad>", cls_token="<s>", sep_token="</s>", mask_token="<mask>")

# Load the dataset
dataset = load_dataset("oscar", "unshuffled_deduplicated_bn", split="train[:1%]")

# Function to format the dataset for training
def format_dataset(examples):
    texts = examples["text"]
    # Tokenize the texts
    tokenized_texts = bangla_tokenizer(texts, padding="max_length", truncation=True, max_length=512)
    return {"input_ids": tokenized_texts["input_ids"], "attention_mask": tokenized_texts["attention_mask"]}

# Apply tokenization to the dataset
dataset = dataset.map(format_dataset, batched=True)

# Remove unnecessary columns
dataset = dataset.remove_columns(["id", "text"])

# Initialize the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)

# Attach LoRA Adapters
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, lora_config)

# Define data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=bangla_tokenizer,
    mlm=False,
    pad_to_multiple_of=8 
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./new_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=1,
    evaluation_strategy="steps",
    eval_steps=10_000,
    learning_rate=5e-5,
    weight_decay=0.01,
    # report_to="wandb",
    remove_unused_columns=False
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=dataset
)

# Train the model with progress bar
for epoch in range(training_args.num_train_epochs):
    print(f"Epoch {epoch+1}/{training_args.num_train_epochs}")
    trainer.train()

# Save the model with progress bar
print("Saving the model...")
trainer.save_model("./new_model")
bangla_tokenizer.save("new_model")
print("Model saved.")

# Finish Weights & Biases run
# wandb.finish()


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.4.
   \\   /|    GPU: NVIDIA A100-SXM4-80GB. Max memory: 79.256 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.2+cu118. CUDA = 8.0. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.23.post1+cu118. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch 1/3
[2024-08-08 15:34:33,813] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 11,145 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 1
\        /    Total batch size = 16 | Total steps = 2,091
 "-____-"     Number of trainable parameters = 6,815,744
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: suvrakamal. Use `wandb login --relogin` to force relogin


...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 11,145 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 1
\        /    Total batch size = 16 | Total steps = 2,091
 "-____-"     Number of trainable parameters = 6,815,744


Epoch 3/3


Step,Training Loss,Validation Loss


Saving the model...


AttributeError: 'PreTrainedTokenizerFast' object has no attribute 'save'

In [4]:
print("Saving the model...")
trainer.save_model("./new_model")
bangla_tokenizer.save_pretrained("new_model")
print("Model saved.")

Saving the model...
Model saved.


In [ ]:
!pip install transformers peft --quiet

In [ ]:

basemodel_path = "unsloth/Meta-Llama-3.1-8B"
loramodel_path = "bangla_llama"
hf_token = "HF_TOKEN"
output_model_name = "bangla_toknizer"

In [ ]:
# Login to Hugging Face
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel
import huggingface_hub as hf_hub

hf_hub.login(hf_token)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(basemodel_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(basemodel_path, trust_remote_code=True, low_cpu_mem_usage=True, torch_dtype=torch.bfloat16)


In [ ]:
# Load and merge LoRA model
if loramodel_path:
    lora_model = PeftModel.from_pretrained(model, loramodel_path)
    model = lora_model.merge_and_unload()

In [ ]:
# Save the tokenizer and model locally
tokenizer.save_pretrained(output_model_name)
model.save_pretrained(output_model_name)

In [ ]:
# Push to Hugging Face Hub
repo = hf_hub.create_repo(output_model_name, private=False)  # Set private=False if you want it to be public
hf_hub.upload_folder(
    folder_path=output_model_name,
    path_in_repo='',  # Root of the repo
    repo_id=f"{hf_hub.whoami()['name']}/{output_model_name}"
)